In [1]:
import pandas as pd
from sentinelsat import SentinelAPI, geojson_to_wkt
import shapely.wkt
from shapely.geometry import Polygon
if __name__ == "__main__" and __package__ is None:
    from sys import path
    from os.path import dirname as dir
    path.append(dir(path[0]))
    __package__ = "examples"
from utils.geospatial_data_utils import GeoTransform, make_rect_poly

### User input

In [2]:
NW = (9.5, 26.5)  # north-west coordinates of AOI box
SE = (7, 28.5)  # south east coordinates of AOI box
CRS = '4326'  # '2154'  # coordinate reference system for AOI

In [3]:
transform = GeoTransform(CRS, '4326', loc2loc=False)

/home/michaeltrs/Programming/miniconda3/envs/satdata/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/home/michaeltrs/Programming/miniconda3/envs/satdata/lib/python3.8/site-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/home/michaeltrs/Programming/miniconda3/envs/satdata/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>

### Make rectangular polygon for AOI extent

In [4]:
NW_glob = transform(NW[1], NW[0])
SE_glob = transform(SE[1], SE[0])
AOI = Polygon([[NW_glob[1], NW_glob[0]],
               [NW_glob[1], SE_glob[0]],
               [SE_glob[1], SE_glob[0]],
               [SE_glob[1], NW_glob[0]],
               [NW_glob[1], NW_glob[0]]])
print('AOI area: ', AOI.area)

AOI area:  5.0


### Query for products

In [5]:
poly = make_rect_poly(NW_glob, SE_glob)
footprint = geojson_to_wkt(poly)
cred = pd.read_csv("pw.csv", header=None)
api = SentinelAPI(cred[0][0], cred[0][1], 'https://scihub.copernicus.eu/dhus')
print("querying...")
products = api.query(footprint,
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0,100),
                     area_relation='Intersects',
                     date=('20200101', '20200201'),
                     processinglevel='Level-1C')

# find unique tiles
tiles = {}
tileids = []
for prod in products:
    if products[prod]['tileid'] not in tileids:
        tileids.append(products[prod]['tileid'])
        tiles[prod] = products[prod]
    # print(products[prod].keys())
    # break
print("found tiles overlapping with AOI: %s" % ", ".join(tileids))

# find overlap with AOI for each tile
print("finding overlap with AOI:")
print("----------------------------------------------")
print("tile id  | AOI/Tile overlap | Tile/AOI overlap")
print("----------------------------------------------")
for i, pr in enumerate(list(tiles.keys())):
    meta = api.get_product_odata(pr)
    tile = shapely.wkt.loads(meta['footprint'])
    aoi_cover_ratio = AOI.intersection(tile).area/AOI.area
    tile_cover_ratio = AOI.intersection(tile).area/tile.area
    print("%s   |      %.4f        |      %.4f"  
          % (tileids[i], aoi_cover_ratio, tile_cover_ratio))

querying...


Querying products: 100%|██████████| 121/121 [00:02<00:00,  9.53 products/s]


found tiles overlapping with AOI: 35PPL, 35NPH, 35NPJ, 35PPK, 35PNK, 35NNH, 35NNJ, 35PNL, 35PML, 35PMK, 35NMJ, 35NMH
finding overlap with AOI:
----------------------------------------------
tile id  | AOI/Tile overlap | Tile/AOI overlap
----------------------------------------------
35PPL   |      0.0391        |      0.2733
35NPH   |      0.0281        |      0.1422
35NPJ   |      0.1178        |      0.5962
35PPK   |      0.1007        |      0.5554
35PNK   |      0.0060        |      1.0000
35NNH   |      0.0150        |      0.1894
35NNJ   |      0.0403        |      1.0000
35PNL   |      0.1084        |      0.5458
35PML   |      0.0638        |      0.3213
35PMK   |      0.1169        |      0.5900
35NMJ   |      0.1169        |      0.5911
35NMH   |      0.0280        |      0.1416
